In [8]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import plotly.express    as px
import random

import plotly.io as pio
pio.renderers.default = 'vscode'

### Control Panel

In [9]:
n_submission = 1000
n_essay = 10

q1_dist = "alluniform"
q2_dist = "uniform"
q3_dist = "uniform"

# q1_dist = "gaussian"
# q2_dist = "gaussian"
# q3_dist = "gaussian"

q1_gaussian_mu = 2.5
q1_gaussian_sigma = 5.
q2_gaussian_mu = 7.
q2_gaussian_sigma = 12.
q3_gaussian_mu = 5.
q3_gaussian_sigma = 1.



### Functions

In [10]:
def generate_random_gaussian_in_range(mean=4.5, std_dev=2., lower_bound=0., upper_bound=9.):
    while True:
        num = np.random.normal(mean, std_dev)
        if lower_bound < num < upper_bound:
            return num

def plot_result(df_in, x_label="x", y_label="y", title=""):

    df = df_in.copy()
    # Add a trendline using numpy
    z = np.polyfit(df['x'], df['y'], 1)  # Linear regression
    p = np.poly1d(z)
    df['trendline'] = p(df['x'])

    fig = px.scatter(df, x='x', y='y', color='y', size='y',
                     labels={'x':x_label, 'y':y_label}, title=title
                     )
    fig.add_scatter(x=df['x'], y=df['trendline'], mode='lines', name='Trendline')
    fig.show()
    return

In [11]:

avg_e = 1.     # Maximum avg_e which means perfect English language
avg_s = 0.2    # Minimum avg_s based on the documentation (avg_s = max(avg_s, 0.2))

final_score_vec = []
avg_h_vec = []
min_v_vec = []
max_final_score = 0
for _ in range(n_submission):
    q1_vec = []
    q2_vec = []
    q3_vec = []
    aqs_vec = []    # avg_quality_scores vector
    hs_vec = []     # horizontal_stdevs vector
    for _ in range(n_essay):
        if q1_dist.lower() == "alluniform":
            q_vec = np.round([random.uniform(0, 9) for _ in range(3)], 2)
        else:
            if q1_dist.lower() == "uniform":
                q1 = np.round(random.uniform(0, 9), 2)
            elif q1_dist.lower() == "gaussian":
                q1 = np.round(generate_random_gaussian_in_range(q1_gaussian_mu, q1_gaussian_sigma), 2)
            
            if q2_dist.lower() == "uniform":
                q2 = np.round(random.uniform(0, 9), 2)
            elif q2_dist.lower() == "gaussian":
                q2 = np.round(generate_random_gaussian_in_range(q2_gaussian_mu, q2_gaussian_sigma), 2)
            
            if q3_dist.lower() == "uniform":
                q3 = np.round(random.uniform(0, 9), 2)
            elif q3_dist.lower() == "gaussian":
                q3 = np.round(generate_random_gaussian_in_range(q3_gaussian_mu, q3_gaussian_sigma), 2)
            
            q_vec = [q1, q2, q3]

        q1_vec.append(q_vec[0])
        q2_vec.append(q_vec[1])
        q3_vec.append(q_vec[2])
        aqs_vec.append(np.round(np.mean(q_vec), 2))
        hs_vec.append(np.round(np.var(q_vec), 2))

    # Calculate vertical_stdevs
    vs1 = np.round(np.var(q1_vec), 2)
    vs2 = np.round(np.var(q2_vec), 2)
    vs3 = np.round(np.var(q3_vec), 2)

    avg_q = np.round(np.mean(aqs_vec), 2)
    avg_h = np.round(np.mean(hs_vec) , 2)
    min_v = min(vs1, vs2, vs3)

    final_score = (avg_h * min_v * avg_e) / (avg_s * (9. - avg_q))
    final_score_vec.append(final_score)
    avg_h_vec.append(avg_h)
    min_v_vec.append(min_v)

    if final_score > max_final_score:
        max_final_score = final_score
        res = pd.DataFrame()
        res["q1"] = q1_vec
        res["q2"] = q2_vec
        res["q3"] = q3_vec
        res["avg_quality_scores"] = aqs_vec
        res["horizontal_stdevs"] = hs_vec

print("Max Score:", max_final_score)


Max Score: 72.93021978021977


In [12]:
tmp = pd.DataFrame()
tmp["x"] = avg_h_vec
tmp["y"] = final_score_vec
tmp.sort_values(by=['x'], inplace=True)
plot_result(tmp, "avg_h", "Final Score", "Score as a function of horizontal variance")

tmp = pd.DataFrame()
tmp["x"] = min_v_vec
tmp["y"] = final_score_vec
tmp.sort_values(by=['x'], inplace=True)
plot_result(tmp, "min_v", "Final Score", "Score as a function of vertical variance")


In [13]:
df = pd.DataFrame()
df["avg_h"] = avg_h_vec
df["min_v"] = min_v_vec
df["final_score"] = final_score_vec

print(max(final_score_vec))

fig = px.scatter_3d(df, x='avg_h', y='min_v', z='final_score',
                    color=df['final_score'], title="3D Scatter Plot")
fig.show()

72.93021978021977


In [14]:
vs1 = np.var(res["q1"])
vs2 = np.var(res["q2"])
vs3 = np.var(res["q3"])

min_v = np.round(min(vs1, vs2, vs3), 2)

print(min_v)

res

7.96


,q1,q2,q3,avg_quality_scores,horizontal_stdevs
0,7.69,0.23,2.33,3.42,9.87
1,5.71,6.41,6.09,6.07,0.08
2,8.20,8.43,2.79,6.47,6.79
3,2.79,7.51,5.82,5.37,3.81
4,7.76,3.25,0.85,3.95,8.21
5,8.47,7.46,5.23,7.05,1.83
6,0.56,8.96,7.11,5.54,12.99
7,4.56,6.90,1.25,4.24,5.37
8,4.74,8.90,8.96,7.53,3.90
9,0.12,2.78,9.00,3.97,13.85
